In [44]:
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bit_api import openBrowser
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import petname

# browser_id = createBrowser()
browser_id = '142fb8b68f2f4f21a1a12b8784611ef3'

# /browser/open 接口会返回 selenium使用的http地址，以及webdriver的path，直接使用即可
res = openBrowser('142fb8b68f2f4f21a1a12b8784611ef3') # 窗口ID从窗口配置界面中复制，或者api创建后返回

In [46]:

# selenium 连接代码
driverPath = res['data']['driver']
debuggerAddress = res['data']['http']

chrome_options = webdriver.ChromeOptions()
chrome_options.add_experimental_option("debuggerAddress", debuggerAddress)

chrome_service = Service(driverPath)
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# 以下为PC模式下，打开baidu，输入 BitBrowser，点击搜索的案例
driver.get('https://www.facebook.com/')

In [47]:
# 等待页面加载并找到按钮元素
reg_button = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.LINK_TEXT, "建立新帳戶"))
)

reg_button.click()


In [48]:

username_input = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.NAME, "reg_email__"))
)
username_input.send_keys('8618661035719')

In [49]:
    
import time
lastname_input = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.NAME, "lastname"))
)
firstname_input = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.NAME, "firstname"))
)
lastname_input.send_keys("王")
time.sleep(3)
firstname_input.send_keys("一玲")

In [50]:
    
password_input = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.NAME, "reg_passwd__"))
)
password_input.send_keys("Qiyu22stom2024@")

In [51]:
year_choose = Select(WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.NAME, "birthday_year"))
))
year_choose.select_by_value('1988')


month_choose = Select(WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.NAME, "birthday_month"))
))
month_choose.select_by_value('5')

day_choose = Select(WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.NAME, "birthday_day"))
))
day_choose.select_by_value('21')

In [52]:

sex_click = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.NAME, "sex"))
)
sex_click.click()

In [53]:

submit_reg = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.NAME, "websubmit"))
)
submit_reg.click()




In [43]:

reg_tip_span = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.CLASS_NAME, "uiHeaderTitle"))
)
tip = reg_tip_span.text
print(tip)




輸入短訊中的確認碼


In [54]:

sms_input = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.ID, "code_in_cliff"))
)
sms_input.send_keys('76965')


In [56]:

sms_submit_btn = WebDriverWait(driver, 30).until(
    EC.presence_of_element_located((By.NAME, "confirm"))
)
sms_submit_btn.click()

In [121]:
import requests

class Sms:
    def __init__(self, server_url, username, password,sid = 72510):
        self.server_url = server_url
        self.username = username
        self.password = password
        self.sid = 72510
        self.token = None
        self.login()

    def login(self):
        url = f"{self.server_url}/sms/?api=login&user={self.username}&pass={self.password}"
        response = requests.get(url)
        data = response.json()
        print(data)
        if data.get('code') == 0:
            self.token = data.get('token')
        else:
            raise Exception(f"Login failed: {data.get('msg')}")

    def get_summary(self):
        url = f"{self.server_url}/sms/?api=getSummary&token={self.token}"
        response = requests.get(url)
        data = response.json()
        print(data)
        if data.get('code') == 0:
            return data
        else:
            raise Exception(f"Failed to get summary: {data.get('msg')}")

    def get_phone(self, isp=None, province=None, ascription=None, paragraph=None, exclude=None, uid=None, author=None):
        url = f"{self.server_url}/sms/?api=getPhone&token={self.token}&sid={self.sid}"
        params = {
            'isp': isp,
            'Province': province,
            'ascription': ascription,
            'paragraph': paragraph,
            'exclude': exclude,
            'uid': uid,
            'author': author
        }
        response = requests.get(url, params={k: v for k, v in params.items() if v is not None})
        data = response.json()
        print(data)
        if data.get('code') == '0':
            phone = data.get('phone')
            return phone
        else:
            raise Exception(f"Failed to get phone: {data.get('msg')}")

    def get_message(self, phone, sid = None):
        if not sid:
            sid = self.sid
        url = f"{self.server_url}/sms/?api=getMessage&token={self.token}&sid={sid}&phone={phone}"
        response = requests.get(url)
        data = response.json()
        print(data)
        if data.get('code') == "0":
            return data
        elif data.get('code') == -1:
            # 等待短信
            if data.get('msg')!='等待':
                raise Exception(f"Failed to get message: {data.get('msg')}")
            return data.get('msg')
        else:
            raise Exception(f"Failed to get message: {data.get('msg')}")

    def get_message_retry(self, phone, timeout=60, interval=5):
        start_time = time.time()
        count = 1
        while time.time() - start_time < timeout:
            url = f"{self.server_url}/sms/?api=getMessage&token={self.token}&sid={self.sid}&phone={phone}"
            response = requests.get(url)
            data = response.json()
            print(f"{phone}-->第{count}次获取手机验证码:{data.get('msg')}")
            if data.get('code') == "0":
                print(data)
                return data
            if data.get('code') == -1 and data.get('msg') != '等待':
                raise Exception(f"Failed to get message: {data.get('msg')}")
            time.sleep(interval)
        print(f"{phone}-->获取不到手机验证码")
        raise TimeoutError("Failed to get message within the specified timeout period")

    def release_phone(self, phone):
        url = f"{self.server_url}/sms/?api=cancelRecv&token={self.token}&sid={self.sid}&phone={phone}"
        response = requests.get(url)
        data = response.json()
        if data.get('code') == 0:
            return data
        else:
            raise Exception(f"Failed to release phone: {data.get('msg')}")

    def release_all_phones(self):
        url = f"{self.server_url}/sms/?api=cancelAllRecv&token={self.token}"
        response = requests.get(url)
        data = response.json()
        if data.get('code') == 0 or data.get('code') == 200:
            return data
        else:
            raise Exception(f"Failed to release all phones: {data.get('msg')}")
        

    def blacklist_phone(self, phone):
        url = f"{self.server_url}/sms/?api=addBlacklist&token={self.token}&sid={self.sid}&phone={phone}"
        response = requests.get(url)
        data = response.json()
        if data.get('code') == 0:
            return data
        else:
            raise Exception(f"Failed to blacklist phone: {data.get('msg')}")

In [122]:

server_url = "https://api.haozhuma.com"
username = "testlinkedin123"
password = "testlinkedin123a"

sms = Sms(server_url, username, password,sid=72510)

try:
    summary = sms.get_summary()
    print("Account Summary:", summary)
    
    phone_data = sms.get_phone()
    print("Phone Data:", phone_data)
    
    # message = sms.get_message(sid=1000, phone=phone_data['phone'])
    # print("Message:", message)
    
except Exception as e:
    print(e)


{'msg': 'success', 'code': 0, 'token': '8cb2f5bde9e25f07ad768e11b6f0bd44b8103653'}
{'msg': None, 'code': 0, 'money': '21.500', 'fanli_rs': 1, 'fanli_je': '0.00', 'num': '50', 'zhekou': '10', 'lx': '开发用户', 'txsr': '0.000', 'ver': '2.0', 'dizhi': 'http://my.haozhuma.com/download/khd/%E8%B1%AA%E7%8C%AA%E7%94%A8%E6%88%B7%E7%AB%AFV2.0.exe', 'log': '豪猪用户端V2.0\r\n\r\n更新级别：推荐\r\n\r\n更新内容：修复卡密市场乱码，增加运营商参数显示', 'khd_qz': '1'}
Account Summary: {'msg': None, 'code': 0, 'money': '21.500', 'fanli_rs': 1, 'fanli_je': '0.00', 'num': '50', 'zhekou': '10', 'lx': '开发用户', 'txsr': '0.000', 'ver': '2.0', 'dizhi': 'http://my.haozhuma.com/download/khd/%E8%B1%AA%E7%8C%AA%E7%94%A8%E6%88%B7%E7%AB%AFV2.0.exe', 'log': '豪猪用户端V2.0\r\n\r\n更新级别：推荐\r\n\r\n更新内容：修复卡密市场乱码，增加运营商参数显示', 'khd_qz': '1'}
{'code': '0', 'msg': '成功', 'sid': '72510', 'shop_name': '72510', 'country_name': '中国大陆', 'country_code': 'cn', 'country_qu': '+86', 'uid': '72510-QLYBLE9R77', 'phone': '18601577281', 'sp': '中国联通', 'phone_gsd': '江苏 无锡'}
Phone Dat

In [124]:

message = sms.get_message(phone=phone_data)
print("Message:", message)
# data = sms.release_phone(phone_data)
# print("release:",data)

{'msg': '等待', 'code': -1, 'time': '2024-07-10 16:21:47'}
Message: 等待


In [125]:
sms.get_message_retry(phone=phone_data)

18601577281-->第1次获取手机验证码:等待
{'msg': '等待', 'code': -1, 'time': '2024-07-10 16:21:47'}
18601577281-->第1次获取手机验证码:等待
{'msg': '等待', 'code': -1, 'time': '2024-07-10 16:21:47'}
18601577281-->第1次获取手机验证码:等待
{'msg': '等待', 'code': -1, 'time': '2024-07-10 16:21:47'}
18601577281-->第1次获取手机验证码:等待
{'msg': '等待', 'code': -1, 'time': '2024-07-10 16:21:47'}
18601577281-->第1次获取手机验证码:等待
{'msg': '等待', 'code': -1, 'time': '2024-07-10 16:21:47'}
18601577281-->第1次获取手机验证码:等待
{'msg': '等待', 'code': -1, 'time': '2024-07-10 16:21:47'}
18601577281-->第1次获取手机验证码:等待
{'msg': '等待', 'code': -1, 'time': '2024-07-10 16:21:47'}
18601577281-->第1次获取手机验证码:等待
{'msg': '等待', 'code': -1, 'time': '2024-07-10 16:21:47'}
18601577281-->第1次获取手机验证码:等待
{'msg': '等待', 'code': -1, 'time': '2024-07-10 16:21:47'}
18601577281-->获取不到手机验证码


TimeoutError: Failed to get message within the specified timeout period